In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import KFold, StratifiedKFold
import warnings
import gc
import time
import sys
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

from catboost import Pool, CatBoostClassifier



In [ ]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

idx = [c for c in train_df.columns if c not in ['ID_code', 'target']]
target = train_df['target']

df_list = [train_df, test_df]
new_df_list = []
new_train_df = pd.DataFrame()
new_train_df['target'] = train_df['target']
new_train_df['ID_code'] = train_df['ID_code']
new_test_df = pd.DataFrame()
new_test_df['ID_code'] = test_df['ID_code']
new_df_list = [new_train_df, new_test_df]
for i in range(0, len(df_list)):        
    for feat in idx:
        mean = df_list[i].loc[:,feat].mean()
        new_df_list[i][feat] = df_list[i][feat]
#         new_df_list[i]["mm_" + feat] = np.round(df_list[i][feat] - mean , 2) 
#         if mean > 0:
#             new_df_list[i]["is_more_" + feat] = np.where( df_list[i][feat] > mean, 10, -10 )
#         if mean < 0: 
#             new_df_list[i]["is_more_" + feat] = np.where( df_list[i][feat] < mean, 10, -10 )
            
features = [c for c in new_train_df.columns if c not in ['ID_code', 'target']]
target = new_train_df['target']

N_FOLDS = 5.0

model = CatBoostClassifier(loss_function="Logloss", eval_metric="AUC", iterations=1500, l2_leaf_reg= 9, learning_rate= 0.1)
#model = CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 1500, l2_leaf_reg= 9, learning_rate= 0.15)


kf = KFold(n_splits=int(N_FOLDS), random_state=2019, shuffle=True)

y_valid_pred = 0 * target
y_test_pred = 0
y_train_pred = 0 #storing results for ensemble

new_train_df.info()

for idx, (train_index, valid_index) in enumerate(kf.split(new_train_df)):
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    X_train, X_valid = new_train_df[features].iloc[train_index,:], new_train_df[features].iloc[valid_index,:]
    _train = Pool(X_train, label=y_train)
    _valid = Pool(X_valid, label=y_valid)
    print( "\nFold ", idx)
    fit_model = model.fit(_train,
                          eval_set=_valid,
                          use_best_model=True,
                          verbose=200
                         )
    pred = fit_model.predict_proba(X_valid)[:,1]
    print( "  auc = ", roc_auc_score(y_valid, pred) )
    y_valid_pred.iloc[valid_index] = pred
    y_test_pred += fit_model.predict_proba(new_test_df[features])[:,1]
    y_train_pred += fit_model.predict_proba(new_train_df[features])[:,1]
y_test_pred /= N_FOLDS
y_train_pred /= N_FOLDS

#save an intermediate submission
sub_df1 = pd.DataFrame({"ID_code":test_df["ID_code"].values})
sub_df1["target"] = y_test_pred
sub_df1.to_csv("submission_catboost.csv", index=False)

ens_cat_df = pd.DataFrame({"ID_code":train_df["ID_code"].values})
ens_cat_df["output"] = y_train_pred
ens_cat_df.to_csv("..input/catboost_output.csv")



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, target to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB

Fold  0
0:	test: 0.5364710	best: 0.5364710 (0)	total: 180ms	remaining: 4m 29s
200:	test: 0.8776868	best: 0.8776868 (200)	total: 36.5s	remaining: 3m 56s
400:	test: 0.8900528	best: 0.8900528 (400)	total: 1m 14s	remaining: 3m 25s
600:	test: 0.8947133	best: 0.8947133 (600)	total: 1m 52s	remaining: 2m 48s
800:	test: 0.8963380	best: 0.8963380 (800)	total: 2m 28s	remaining: 2m 9s
1000:	test: 0.8966581	best: 0.8967353 (993)	total: 3m 5s	remaining: 1m 32s
1200:	test: 0.8970510	best: 0.8970800 (1194)	total: 3m 41s	remaining: 55.1s
1400:	test: 0.8970907	best: 0.8972818 (1294)	total: 4m 18s	remaining: 18.3s
1499:	test: 0.8971109	best: 0.8972818 (1294)	total: 4m 36s	remaining: 0us

bestTest = 0.8972817673
bestIteration = 1294

Shrink model to first 1295 iterations.
  auc =  0.8972817673087907

Fold  1
0:	test: 0.5289